In [1]:
from __future__ import absolute_import, division, print_function

import argparse
from datetime import datetime
import numpy as np
import os

from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
import numpy as np
from sklearn.metrics import roc_auc_score

from data_handler_gesture_2 import DataHandler
#from data_handler_gesture import DataHandler
from models import create_grud_model, load_grud_model
from nn_utils.callbacks import ModelCheckpointwithBestWeights


Using TensorFlow backend.


In [2]:
# set GPU usage for tensorflow backend
if K.backend() == 'tensorflow':
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = .1
    config.gpu_options.allow_growth = True
    K.set_session(tf.Session(config=config))

In [3]:
# parse arguments
## general
arg_parser = argparse.ArgumentParser()
arg_parser.add_argument('--working_path', default='.')

## data
arg_parser.add_argument('dataset_name', default='mimic3',
                        help='The data files should be saved in [working_path]/data/[dataset_name] directory.')
arg_parser.add_argument('label_name', default='mortality')
arg_parser.add_argument('--max_timesteps', type=int, default=200, 
                        help='Time series of at most # time steps are used. Default: 200.')
arg_parser.add_argument('--max_timestamp', type=int, default=48*60*60,
                        help='Time series of at most # seconds are used. Default: 48 (hours).')

## model
arg_parser.add_argument('--recurrent_dim', type=lambda x: x and [int(xx) for xx in x.split(',')] or [], default='64')
arg_parser.add_argument('--hidden_dim', type=lambda x: x and [int(xx) for xx in x.split(',')] or [], default='64')
arg_parser.add_argument('--model', default='GRUD', choices=['GRUD', 'GRUforward', 'GRU0', 'GRUsimple'])
arg_parser.add_argument('--use_bidirectional_rnn', default=False)
                           
## training
arg_parser.add_argument('--pretrained_model_file', default=None,
                        help='If pre-trained model is provided, training will be skipped.') # e.g., [model_name]_[i_fold].h5
arg_parser.add_argument('--epochs', type=int, default=100)
arg_parser.add_argument('--early_stopping_patience', type=int, default=10)
arg_parser.add_argument('--batch_size', type=int, default=2)


## set the actual arguments if running in notebook
if not (__name__ == '__main__' and '__file__' in globals()):
    '''ARGS = arg_parser.parse_args([
        'mimic3',
        'mortality',
        '--model', 'GRUD',
        '--hidden_dim', '',
        '--epochs', '100'
    ])'''
    ARGS = arg_parser.parse_args([
        'gesture',
        'phase',
        '--model', 'GRUD',
        '--hidden_dim', '',
        '--max_timestamp', '5807537',
        '--epochs', '100'
    ])
else:
    ARGS = arg_parser.parse_args()

print('Arguments:', ARGS)

Arguments: Namespace(batch_size=2, dataset_name='gesture', early_stopping_patience=10, epochs=100, hidden_dim=[], label_name='phase', max_timestamp=5807537, max_timesteps=200, model='GRUD', pretrained_model_file=None, recurrent_dim=[64], use_bidirectional_rnn=False, working_path='.')


In [4]:
print(ARGS.label_name)

phase


In [5]:
print(ARGS.dataset_name)

gesture


In [6]:
# get dataset

dataset = DataHandler(
    data_path=os.path.join(ARGS.working_path, 'data', ARGS.dataset_name), 
    label_name=ARGS.label_name, 
    max_steps=ARGS.max_timesteps,
    max_timestamp=ARGS.max_timestamp
)

self._data['fold'].shape:(7, 5, 3)
self._input_dim:18
self._output_dim:1


In [7]:
# k-fold cross-validation
pred_y_list_all = []
auc_score_list_all = []

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S_%f')
print('Timestamp: {}'.format(timestamp))

for i_fold in range(dataset.folds):
    print('{}-th fold...'.format(i_fold))

    # Load or train the model.
    if ARGS.pretrained_model_file is not None:
        model = load_grud_model(os.path.join(ARGS.working_path, 
                                             ARGS.pretrained_model_file.format(i_fold=i_fold)))
    else:
        model = create_grud_model(input_dim=dataset.input_dim,
                                  output_dim=dataset.output_dim,
                                  output_activation=dataset.output_activation,
                                  recurrent_dim=ARGS.recurrent_dim,
                                  hidden_dim=ARGS.hidden_dim,
                                  predefined_model=ARGS.model,
                                  use_bidirectional_rnn=ARGS.use_bidirectional_rnn
                                 )
        if i_fold == 0:
            model.summary()
        model.compile(optimizer='adam', loss=dataset.loss_function)
        model.fit_generator(
            generator=dataset.training_generator(i_fold, batch_size=ARGS.batch_size),
            steps_per_epoch=dataset.training_steps(i_fold, batch_size=ARGS.batch_size),
            epochs=ARGS.epochs,
            verbose=2,
            validation_data=dataset.validation_generator(i_fold, batch_size=ARGS.batch_size),
            validation_steps=dataset.validation_steps(i_fold, batch_size=ARGS.batch_size),
            callbacks=[
                EarlyStopping(patience=ARGS.early_stopping_patience),
                ModelCheckpointwithBestWeights(
                    file_dir=os.path.join(ARGS.working_path, 'model', timestamp + '_' + str(i_fold))
                ),
                TensorBoard(
                    log_dir=os.path.join(ARGS.working_path, 'tb_logs', timestamp + '_' + str(i_fold))
                )
            ]
            )
        model.save(os.path.join(ARGS.working_path, 'model', 
                                timestamp + '_' + str(i_fold), 'model.h5'))

    # Evaluate the model
    true_y_list = [
        dataset.training_y(i_fold), dataset.validation_y(i_fold), dataset.testing_y(i_fold)
    ]
    pred_y_list = [
        model.predict_generator(dataset.training_generator_x(i_fold, batch_size=ARGS.batch_size),
                                steps=dataset.training_steps(i_fold, batch_size=ARGS.batch_size)),
        model.predict_generator(dataset.validation_generator_x(i_fold, batch_size=ARGS.batch_size),
                                steps=dataset.validation_steps(i_fold, batch_size=ARGS.batch_size)),
        model.predict_generator(dataset.testing_generator_x(i_fold, batch_size=ARGS.batch_size),
                                steps=dataset.testing_steps(i_fold, batch_size=ARGS.batch_size)),
    ]
    auc_score_list = [roc_auc_score(ty, py) for ty, py in zip(true_y_list, pred_y_list)] # [3, n_task]
    print('AUC score of this fold: {}'.format(auc_score_list))
    pred_y_list_all.append(pred_y_list)
    auc_score_list_all.append(auc_score_list)

print('Finished!', '='*20)
auc_score_list_all = np.stack(auc_score_list_all, axis=0)
print('Mean AUC score: {}; Std AUC score: {}'.format(
    np.mean(auc_score_list_all, axis=0),
    np.std(auc_score_list_all, axis=0)))

result_path = os.path.join(ARGS.working_path, 'results', timestamp)
if not os.path.exists(result_path):
    os.makedirs(result_path)
np.savez_compressed(os.path.join(result_path, 'predictions.npz'),
                    pred_y_list_all=pred_y_list_all)
np.savez_compressed(os.path.join(result_path, 'auroc_score.npz'),
                    auc_score_list_all=auc_score_list_all)


Timestamp: 20200114_092005_949463
0-th fold...
input_dim 18
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 18)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 18)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, 1)      0                                            
__________________________________________________________________________________________________
external_masking_1 (ExternalMas (None, None, 18)     0           input_1[0][0]                    
                                                 

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 7 arrays: [array([['Rest'],
       ['Preparation'],
       ['Stroke'],
       ['Stroke'],
       ['Preparation'],
       ['Stroke'],
       ['Retraction'],
       ['Rest']], dtype=object), array([['Rest'],
    ...

In [17]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 200
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32


In [18]:

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')


Loading data...
25000 train sequences
25000 test sequences


In [19]:

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train[:200], maxlen=maxlen)
x_test = sequence.pad_sequences(x_test[:200], maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


Pad sequences (samples x time)
x_train shape: (200, 80)
x_test shape: (200, 80)


In [21]:

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train[:200], y_train[:200],
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test[:200], y_test[:200]))
score, acc = model.evaluate(x_test[:200], y_test[:200],
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Build model...
Train...
Train on 200 samples, validate on 200 samples
Epoch 1/15
200/200 [==============================] - 3s 14ms/step - loss: 0.6889 - acc: 0.5700 - val_loss: 0.6904 - val_acc: 0.5500
Epoch 2/15
200/200 [==============================] - 1s 7ms/step - loss: 0.6771 - acc: 0.5750 - val_loss: 0.6912 - val_acc: 0.5500
Epoch 3/15
200/200 [==============================] - 1s 7ms/step - loss: 0.6714 - acc: 0.5750 - val_loss: 0.6940 - val_acc: 0.5500
Epoch 4/15
200/200 [==============================] - 1s 7ms/step - loss: 0.6577 - acc: 0.5750 - val_loss: 0.6919 - val_acc: 0.5500
Epoch 5/15
200/200 [==============================] - 1s 7ms/step - loss: 0.6442 - acc: 0.5900 - val_loss: 0.6908 - val_acc: 0.5550
Epoch 6/15
200/200 [==============================] - 1s 7ms/step - loss: 0.6129 - acc: 0.7250 - val_loss: 0.6894 - val_acc: 0.5450
Epoch 7/15
200/200 [==============================] - 1s 7ms/step - loss: 0.5868 - acc: 0.7450 - val_loss: 0.7015 - val_acc: 0.5600
Epoch

In [33]:
pred = model.predict(x_test[:20],verbose=1)

20/20 [==============================] - 0s 2ms/step


In [37]:
pred.shape

(20, 1)

In [38]:
np.argmax(pred)

12

In [36]:
words

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,